In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('data/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

incomplete observations:

 3   LotFrontage    1201 non-null   float64

 6   Alley          91 non-null     object 

 30  BsmtQual       1423 non-null   object 

 31  BsmtCond       1423 non-null   object 

 32  BsmtExposure   1422 non-null   object 

 33  BsmtFinType1   1423 non-null   object 

 35  BsmtFinType2   1422 non-null   object 

 42  Electrical     1459 non-null   object 

 57  FireplaceQu    770 non-null    object 

 58  GarageType     1379 non-null   object 

 59  GarageYrBlt    1379 non-null   float64

 60  GarageFinish   1379 non-null   object

 72  PoolQC         7 non-null      object 

 73  Fence          281 non-null    object 
 
 74  MiscFeature    54 non-null     object

In [12]:
# Should be categorical type
df['MSSubClass'] = df.MSSubClass.astype(str)
df.MSSubClass.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1460 entries, 0 to 1459
Series name: MSSubClass
Non-Null Count  Dtype 
--------------  ----- 
1460 non-null   object
dtypes: object(1)
memory usage: 11.5+ KB


In [16]:
# Fix LotFrontage
df.LotFrontage.fillna(0, inplace=True)
df.LotFrontage.value_counts()

0.0      259
60.0     143
70.0      70
80.0      69
50.0      57
        ... 
137.0      1
38.0       1
33.0       1
150.0      1
46.0       1
Name: LotFrontage, Length: 111, dtype: int64

In [8]:
# Should be NA not NAN
df.Alley.fillna('NA', inplace=True)
df.Alley.value_counts()

NA      1369
Grvl      50
Pave      41
Name: Alley, dtype: int64

In [30]:
# Should be NA not NAN
df.BsmtQual.fillna('NA', inplace=True)
df.BsmtCond.fillna('NA', inplace=True)
df.BsmtExposure.fillna('NA', inplace=True)
df.BsmtFinType1.fillna('NA', inplace=True)
df.BsmtFinType2.fillna('NA', inplace=True)
df.BsmtQual.value_counts()

TA    649
Gd    618
Ex    121
NA     37
Fa     35
Name: BsmtQual, dtype: int64

In [31]:
df.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [33]:
# Unsure how to replace electrical value, so we'll drop the observation with the missing value
len(df)

1459

In [34]:
# Fireplace should be NA not NAN
df.FireplaceQu.fillna('NA', inplace=True)
df.FireplaceQu.value_counts()

NA    689
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

In [ ]:
df.GarageType.fillna('NA', inplace=True)
df.GarageYrBlt.fillna(-1, inplace=True)
df